# Document Question Answering Chatbot

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [14]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

open.api_key = os.environ['OPENAI_API_KEY']

In [15]:
def ask_question_with_context(qa, question, chat_history):
    result = qa({"question": question, "chat_history": chat_history})
    print(f"\n{textwrap.fill(result['answer'])}\n")
    chat_history = [(question, result["answer"])]
    return chat_history

## Initialize ChromaDB

In [16]:
embeddings = OpenAIEmbeddings()
persist_directory = 'epiq_policies_chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

255


## Create the chain and a helper function

Initialize the chain we will use for question answering.

In [8]:
#llm_model_name = "gpt-3.5-turbo"
llm_model_name = "gpt-4"
llm = ChatOpenAI(model_name=llm_model_name, temperature=0)

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [9]:
QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:""")

In [10]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_prompt=QUESTION_PROMPT,
    return_source_documents=True,
    verbose=False)

In [11]:
chat_history = []

## Ask questions!

Now we can use the chain to ask questions!

In [12]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    chat_history = ask_question_with_context(qa, query, chat_history)

you:  what can you tell me about the question generation paper?


AuthenticationError: Incorrect API key provided: sk-5tdVr***************************************mAqf. You can find your API key at https://platform.openai.com/account/api-keys.